### Gmarket
- 베스트 상품 200개 데이터 수집
- 상품의 이미지 200개 다운로드

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### 1. URL 찾기

In [3]:
url = 'https://corners.gmarket.co.kr/bestsellers'


#### 2. request > response : str(html)

In [4]:
response = requests.get(url)
response

<Response [200]>

In [10]:
dom = BeautifulSoup(response.text, "html.parser")
elements = dom.select('#gBestWrap > div > div:nth-child(5) > div > ul > li')

In [12]:
len(elements)

200

In [14]:
element = elements[0]

In [32]:
data = {
    'tilte':element.select_one('.itemname').text,
    'link' :element.select_one('.itemname').get('href'),
    'image' :'https://'+element.select_one('img').get('data-original'),
    'o-price' :element.select_one('.o-price').text,
    's-price' :element.select_one('.s-price').text.strip().split('\n')[0],
}
data

{'tilte': '[매일유업]매일 상하멸균우유/케일주스/요미요미24팩모음',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=2628342593&ver=638122426590921333',
 'image': 'https:////gdimg.gmarket.co.kr/2628342593/still/300?ver=2023021714',
 'o-price': '정가33,800원',
 's-price': '할인가23,700원'}

In [84]:
data = []
for element in elements:
    data.append({
    'tilte':element.select_one('.itemname').text,
    'link' :element.select_one('.itemname').get('href'),
    'image' :'http:'+element.select_one('img').get('data-original'),
    'o-price' :element.select_one('.o-price').text,
    's-price' :element.select_one('.s-price').text.strip().split('\n')[0],
})

#### 3. bs > DataFrame

In [85]:
df = pd.DataFrame(data)
df.tail(2)

,tilte,link,image,o-price,s-price
198,레인보우 중목 라인양말_10+10켤레 세트 (할인특가),http://item.gmarket.co.kr/Item?goodscode=27936...,http://gdimg.gmarket.co.kr/2793644100/still/30...,"정가43,000원","할인가12,900원"
199,[길벗스쿨]길벗스쿨_언어천재 말놀이 세트(본책9권+꾸러미북1권+스티커북1권+말놀이 ...,http://item.gmarket.co.kr/Item?goodscode=27917...,http://gdimg.gmarket.co.kr/2791705618/still/30...,"정가96,000원","할인가80,000원"


In [86]:
idx = df[df['o-price'] == ''].index

In [87]:
df['o-price'].loc[idx] = df['s-price'].loc[idx]

In [88]:
df[0:5]

,tilte,link,image,o-price,s-price
0,[매일유업]매일 상하멸균우유/케일주스/요미요미24팩모음,http://item.gmarket.co.kr/Item?goodscode=26283...,http://gdimg.gmarket.co.kr/2628342593/still/30...,"정가33,800원","할인가23,700원"
1,[피셔프라이스]피셔프라이스 러닝 푸드트럭,http://item.gmarket.co.kr/Item?goodscode=19985...,http://gdimg.gmarket.co.kr/1998535842/still/30...,"정가210,000원","할인가99,000원"
2,[나이키키즈](대구신세계)[나이키키즈] (105~170) 조던 썸머 상/하의세트 세일가,http://item.gmarket.co.kr/Item?goodscode=27728...,http://gdimg.gmarket.co.kr/2772877848/still/30...,"정가58,000원","할인가45,820원"
3,(지오지아)사계절 테이퍼드/와이드핏 슬랙스 치노팬츠 10종 택1,http://item.gmarket.co.kr/Item?goodscode=27678...,http://gdimg.gmarket.co.kr/2767818438/still/30...,"정가32,000원","할인가24,640원"
4,[플랙]남성 데님 팬츠 6종 택1(PJTT5MLD731외5종),http://item.gmarket.co.kr/Item?goodscode=27808...,http://gdimg.gmarket.co.kr/2780877061/still/30...,"정가69,000원","할인가50,370원"


In [89]:
import re

In [90]:
df['o-price'] = df['o-price'].apply(lambda price: re.findall('[0-9,]+',price)[0])
df['s-price'] = df['s-price'].apply(lambda price: re.findall('[0-9,]+',price)[0])
df.tail(4)

,tilte,link,image,o-price,s-price
196,[압타밀]압타밀 프로푸트라 1단계 800g 4통 세트 독일내수용 / 빠른배송 / 최신제품,http://item.gmarket.co.kr/Item?goodscode=27748...,http://gdimg.gmarket.co.kr/2774807080/still/30...,"149,800","143,060"
197,2세대 코코맘 이동식트롤리 기저귀정리함 3단 캡형,http://item.gmarket.co.kr/Item?goodscode=27623...,http://gdimg.gmarket.co.kr/2762398385/still/30...,"51,900","44,900"
198,레인보우 중목 라인양말_10+10켤레 세트 (할인특가),http://item.gmarket.co.kr/Item?goodscode=27936...,http://gdimg.gmarket.co.kr/2793644100/still/30...,"43,000","12,900"
199,[길벗스쿨]길벗스쿨_언어천재 말놀이 세트(본책9권+꾸러미북1권+스티커북1권+말놀이 ...,http://item.gmarket.co.kr/Item?goodscode=27917...,http://gdimg.gmarket.co.kr/2791705618/still/30...,"96,000","80,000"


In [91]:
# df.str.replace('할인가', '')
# df.str.replace('정가', '')
# df.str.replace('원', '')
price = '할인가1,111,800원'
re.findall('[0-9,]+',price)[0]

'1,111,800'

In [92]:
# re(정규표현식) 0~9 문자가 한 개 이상 있을 경우 선택,
txt = '저는 정재훈입니다 이메일은 sagvd@naver.com op6160@gmail.com 이에요'
re.findall('[a-zA-z0-9.]+@[a-z]+\.[a-z]+',txt)

['sagvd@naver.com', 'op6160@gmail.com']

#### 4. download image

In [93]:
# 이미지 파일 저장 디렉토리
import os
if not os.path.exists('data'):
    os.makedirs('data')

In [101]:
image_link = df.loc[0,'image']
rseponse = requests.get(image_link)
response
image_link

'http://gdimg.gmarket.co.kr/2628342593/still/300?ver=2023021714'

In [113]:
for i,v in df[:5].iterrows():
    filename = '0'*(2-len(str(i)))+str(i)
    filename = f'data/{filename}.png' 
    rseponse = requests.get(v['image'])
    with open(filename,'wb') as file:
        file.write(response.content)

In [115]:
from PIL import Image as pil
#pil.open('data/test.png')